In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random
import os
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import TimeoutException
import easyocr

In [ ]:
class SaraminCrawler:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # 섹션별 키워드 매핑
        self.section_patterns = {
            '주요업무': [
                r'📋\s*주요업무(.*?)(?=📋|🏠|🎁|🚀|$)',
                r'■\s*주요\s*업무(.*?)(?=■|$)',
                r'■\s*담당\s*업무(.*?)(?=■|$)',
                r'주요\s*업무\s*[:\n]+(.*?)(?=자격|우대|근무|복지|채용|$)',
            ],
            '자격요건': [
                r'📋\s*자격요건(.*?)(?=📋|🏠|🎁|🚀|$)',
                r'■\s*자격\s*요건(.*?)(?=■|$)',
                r'자격\s*요건\s*[:\n]+(.*?)(?=우대|근무|복지|채용|$)',
            ],
            '우대사항': [
                r'📋\s*우대사항(.*?)(?=📋|🏠|🎁|🚀|$)',
                r'■\s*우대\s*사항(.*?)(?=■|$)',
                r'우대\s*사항\s*[:\n]+(.*?)(?=근무|복지|채용|$)',
            ],
            '채용절차': [
                r'🚀\s*채용절차(.*?)(?=🛎️|$)',
                r'전형\s*절차\s*[:\n]+(.*?)(?=유의|$)',
                r'채용\s*절차.*?[:\n]+(.*?)(?=유의|제출|$)',
            ],
            '복지 및 혜택': [
                r'🎁\s*복지\s*및\s*혜택(.*?)(?=🚀|$)',
                r'복지\s*및\s*혜택\s*[:\n]+(.*?)(?=채용|근무|$)',
                r'복리후생(.*?)(?=채용|근무|$)',
            ],
            '급여': [
                r'급여\s*[:：]\s*(.+?)(?=\n|근무|$)',
                r'연봉\s*[:：]\s*(.+?)(?=\n|근무|$)',
            ]
        }

    

    def clean_text(self, text):
        """텍스트 정제"""
        if not text:
            return ""
        # 이모지 및 특수문자 제거
        text = re.sub(r'[📋🏠🎁🚀🛎️ㆍ•·]', '', text)
        # HTML 태그 제거
        text = re.sub(r'<[^>]+>', '', text)
        # 연속 줄바꿈을 공백으로
        text = re.sub(r'\n+', ' ', text)
        # 연속 공백 정리
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def extract_bullet_points(self, text):
        """불릿 포인트 항목 추출"""
        # 다양한 불릿 포인트 패턴
        patterns = [
            r'[-−–—]\s*(.+?)(?=[-−–—]|\n\n|$)',
            r'[•∙·]\s*(.+?)(?=[•∙·]|\n\n|$)',
            r'\d+[.)]\s*(.+?)(?=\d+[.)]|\n\n|$)',
        ]
        items = []
        for pattern in patterns:
            matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL)
            for match in matches:
                cleaned = self.clean_text(match)
                if len(cleaned) > 5:  # 너무 짧은 것 제외
                    items.append(cleaned)
        return items
    
    def extract_section(self, markdown_text, section_name):
        """특정 섹션 추출"""
        patterns = self.section_patterns.get(section_name, [])
        for pattern in patterns:
            match = re.search(pattern, markdown_text, re.DOTALL | re.IGNORECASE)
            if match:
                content = match.group(1).strip()
                # 불릿 포인트 추출 시도
                items = self.extract_bullet_points(content)
                if items:
                    return ', '.join(items)
                # 불릿 포인트가 없으면 전체 텍스트 정제
                return self.clean_text(content)
        return ""
    
    def extract_all_sections(self, markdown_text):
        """모든 섹션 추출"""
        result = {}
        for section_name in ['주요업무', '자격요건', '우대사항', '채용절차', '복지 및 혜택', '급여']:
            result[section_name] = self.extract_section(markdown_text, section_name)
        return result
    
    def search_jobs(self, keyword=None, **filters):
        """실제 api 엔드포인트 사용한 검색"""

        jobs = []
        curren_page = 6

        # 실제 API URL
        api_url = "https://www.saramin.co.kr/zf_user/search/get-recruit-list"

        # API 파라미터 (Network 탭에서 발견한 것들)
        params = {
            'searchType': 'search',
            'recruitPage': curren_page,     # 페이지네이션
            'recruitSort': 'relation',   # 정렬기준(관련도순)
            'recruitPageCount': 40,    # 한 폐이지에 보이는 개수
            'search_optional_item': 'y',
            'search_done': 'y',
            'panel_count': 'y',
            'preview': 'y',
            'mainSearch': 'n'
        }

        # 검색어
        if keyword:
            params['searchword'] = keyword

        # 고급 필터 적용
        #self._apply_filters(params, filters)

        try:
            response = requests.get(api_url, params=params, headers=self.headers)
            response.raise_for_status()

            # JSON 응답 파싱 : 응답은 json 형태이기때문 '{"count":"283","innerHTML":"<div>...</div>"}'
            json_data = response.json()

            # 전체 공고 수 계산
            total_count = int(json_data.get('count', '0').replace(',', ''))

            # 공고가 많으면 5페이지만 크롤링 
            max_pages = min((total_count + 39) // 40, 1)

            print(f"총 {total_count:,}개 공고 발견! {max_pages}페이지 크롤링 예정")
            wait = 1.0
            for page in range(1, max_pages + 1):
                
                print(f"📄 {page}/{max_pages} 페이지 수집 중...")

                params['recruitPage'] = page

                try: 
                    # 각 페이지마다 새로 API 호출
                    response = requests.get(api_url, params=params, headers=self.headers)
                    response.raise_for_status()
                    json_data = response.json()  # 해당 페이지 데이터

                    if json_data.get('innerHTML'):
                        # 채용공고 추출
                        soup = BeautifulSoup(json_data['innerHTML'], 'html.parser')
                        json_itmes = soup.find_all('div', class_='item_recruit')
                        
                        if not json_itmes:
                            print(f"페이지 {page}에서 공고를 찾을 수 없습니다.")
                            break

                        print(f"총 {len(json_itmes)}개 수집")

                        row_count = 0

                        for item in json_itmes:
                            job_data = self.extract_job_info_from_api(item, keyword or '전체')
                            if job_data:
                                row_count += 1
                                time.sleep(random.uniform(1.5, 4.5))
                                self.detail_webdriver_search(job_data)
                                jobs.append(job_data)
                                
                                print(f"{row_count}번째 수집완료!!")
                                
                    else:
                        print(f"페이지 {page}에서 데이터를 받지 못했습니다.")
                        break

                    time.sleep(random.uniform(1.5, 4.5))  # 서버 부하 및 로못 방지 방지

                except Exception as e:
                    print(f"페이지 {page} 크롤링 실패: {e}")
                    continue
        
        except Exception as e:
            print(f"초기 데이터 로딩 실패: {e}")
            return []
        
        print(f"✅ '{keyword or '전체'}' 총 {len(jobs)}개 공고 수집 완료!")

        df = pd.DataFrame(jobs)
        df.to_csv(
            f'../data/사람인_{keyword} 채용공고_{curren_page}.csv'
        )
        
        # return jobs
    
    def extract_job_info_from_api(self, item, keyword):
        """API에서 받은 HTML 구조에 맞게 정보 추출"""
        try:
            # 공고 제목 및 링크
            title_elem = item.select_one('div.area_job > h2.job_tit > a')
            title = title_elem.get_text(strip=True) if title_elem else print("공고명 못찾음❌❌")

            # 링크 처리 (상대경로 → 절대경로 변환)
            href = title_elem.get('href') if title_elem else ""
            link = f"https://www.saramin.co.kr{href}" if href else ""
            
            # 회사명
            company_elem = item.select_one('div.area_corp > strong.corp_name > a')
            company = company_elem.get_text(strip=True) if company_elem else print("회사명 찾지 못함❌❌")

            # 마감일
            deadline_elem = item.select_one('div.area_job > div.job_date > span.date')
            deadline = deadline_elem.get_text(strip=True) if deadline_elem else print("마감일 찾지 못함❌❌")

            # 위치, 경력 정보
            condition_elem = item.select('div.area_job > div.job_condition > span')

            # 키워드1
            keyword_elem = item.select('div.area_job > div.job_sector > b > a')
            # 키워드2
            keyword_elem2 = item.select('div.area_job > div.job_sector > a')

            # 기본값 설정
            location = "지역 없음"  
            career = "경력 없음"
            education = "학력 없음"  
            work_type = "근무형태 없음"
            keyword_text = "키워드 없음"

            # 안전하게 인덱스 확인 후 추출
            if len(condition_elem) > 0:
                location_elem = condition_elem[0].select('a')
                location_list = [loc.get_text(strip=True) for loc in location_elem]
    
                if len(location_list) >= 2:
                    # "서울 강남구" 형태로
                    location = " ".join(location_list)
                elif len(location_list) == 1:
                    # "서울" 같이 하나만 있는 경우
                    location = location_list[0]
                else:
                    location = "지역 없음"

            # 경력 
            if len(condition_elem) > 1:
                career_elem = condition_elem[1]
                career = career_elem.get_text(strip=True)

            # 학력
            if len(condition_elem) > 2:
                edu_elem = condition_elem[2]
                education = edu_elem.get_text(strip=True)

            # 근무 조건
            if len(condition_elem) > 3:
                work_type_elem = condition_elem[3]
                work_type = work_type_elem.get_text(strip=True)

            # 공고 ID 추출
            rec_idx = item.get('value', '')

            if len(keyword_elem) > 1:

                keyword_list = [loc.get_text(strip=True) for loc in keyword_elem]
                if len(keyword_elem) == 1:
                    keyword_text = keyword_list[0]
                else:
                    keyword_text = " ".join(keyword_list)
                
            if len(keyword_elem2) > 1:
                keyword_list2 = [loc.get_text(strip=True) for loc in keyword_elem2]
                if len(keyword_elem) > 1:
                    keyword_text += " " + " ".join(keyword_list2)
                else:
                    if len(keyword_elem2) == 1:
                        keyword_text = keyword_list2[0]
                    else:
                        keyword_text = " ".join(keyword_list2)

            return {
                'keyword': keyword,
                'title': title,
                'company': company,
                'location': location,
                'career': career,
                'education': education,
                'work_type': work_type,
                'deadline': deadline,
                'link': link,
                'rec_idx': rec_idx,
                'keyword_text': keyword_text,
                'crawled_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
        except Exception as e:
            print(f"공고 정보 추출 실패 : {e}")
            print(f"문제 공고 HTML 구조: {item}")
            return None

    def image_to_text(self, img_url):
        reader = easyocr.Reader(['ko', 'en'], gpu=False)
        result = reader.readtext(img_url, detail=0)    
        result_text = ""
        for text in result:
            result_text += text + " "

        return result_text
    
    def detail_webdriver_search(self, detail_param):
        driver = webdriver.Chrome()
        try:
            wait = WebDriverWait(driver, 20)
            link_url = f"https://m.saramin.co.kr/job-search/view?rec_idx={detail_param['rec_idx']}"
            driver.get(link_url)    

            
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[id^='iframe_recruit_detail_'], iframe[src*='view-frame']")))
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detailContentsWrap")))
            job_content = driver.find_element(By.CSS_SELECTOR, ".detailContentsWrap").text

            img_contents = driver.find_elements(By.CSS_SELECTOR, ".image > img")
            img_text = ""
            if len(img_contents) > 0:
                for img_content in img_contents:
                    img_url=img_content.get_attribute("src")
                    img_text = self.image_to_text(img_url)
                    job_content += " " + img_text  

            driver.switch_to.default_content()  # 찾아가기 초기화
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detail_corp")))

            company_content = driver.find_element(By.CSS_SELECTOR, ".detail_corp")
            company_items = company_content.find_elements(By.CSS_SELECTOR, ".detail_item > dd")[2].text
            content_dict = self.extract_all_sections(job_content)

            detail_param["responsibilities"] = content_dict["주요업무"]
            detail_param["qualifications"] = content_dict["자격요건"]
            detail_param["preferred_quali"] = content_dict["우대사항"]
            detail_param["hiring_process"] = content_dict["채용절차"]
            detail_param["benefits"] = content_dict["복지 및 혜택"]
            detail_param["salary"] = content_dict["급여"]
            detail_param["corp_domain"] = company_items
            detail_param["totaltext"] = job_content

            driver.close()

            return(detail_param)
        
        except Exception as e:
            print(f"공고 정보 추출 실패 : {e}")
            print(f"상세페이지 id: {detail_param['rec_idx']}")
            return None
        except TimeoutException as e:
            print(f"공고 정보 추출 실패 : {e}")
            print(f"상세페이지 id: {detail_param['rec_idx']}")
            return None
    

self = SaraminCrawler()

self.search_jobs("AI 에이전트")
#self.detail_info_job_api()

총 480개 공고 발견! 1페이지 크롤링 예정
📄 1/1 페이지 수집 중...
총 40개 수집
1번째 수집완료!!


Using CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Using CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 101.9% Complete3번째 수집완료!!


...

Progress: |██████████████████████████████████████████████████| 100.3% Complete

c:\Source\TEAM-PJ-01\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


25번째 수집완료!!


Using CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████████████| 116.1% Complete26번째 수집완료!!


In [ ]:

section_patterns = {
    '주요업무': [
        r'📋\s*주요업무(.*?)(?=📋|🏠|🎁|🚀|$)',
        r'■\s*주요\s*업무(.*?)(?=■|$)',
        r'■\s*담당\s*업무(.*?)(?=■|$)',
        r'주요\s*업무\s*[:\n]+(.*?)(?=자격|우대|근무|복지|채용|$)',
    ],
    '자격요건': [
        r'📋\s*자격요건(.*?)(?=📋|🏠|🎁|🚀|$)',
        r'■\s*자격\s*요건(.*?)(?=■|$)',
        r'자격\s*요건\s*[:\n]+(.*?)(?=우대|근무|복지|채용|$)',
    ],
    '우대사항': [
        r'📋\s*우대사항(.*?)(?=📋|🏠|🎁|🚀|$)',
        r'■\s*우대\s*사항(.*?)(?=■|$)',
        r'우대\s*사항\s*[:\n]+(.*?)(?=근무|복지|채용|$)',
    ],
    '채용절차': [
        r'🚀\s*채용절차(.*?)(?=🛎️|$)',
        r'전형\s*절차\s*[:\n]+(.*?)(?=유의|$)',
        r'채용\s*절차.*?[:\n]+(.*?)(?=유의|제출|$)',
    ],
    '복지 및 혜택': [
        r'🎁\s*복지\s*및\s*혜택(.*?)(?=🚀|$)',
        r'복지\s*및\s*혜택\s*[:\n]+(.*?)(?=채용|근무|$)',
        r'복리후생(.*?)(?=채용|근무|$)',
    ],
    '급여': [
        r'급여\s*[:：]\s*(.+?)(?=\n|근무|$)',
        r'연봉\s*[:：]\s*(.+?)(?=\n|근무|$)',
    ]
}

def clean_text(text):
    """텍스트 정제"""
    if not text:
        return ""
    
    # 이모지 및 특수문자 제거
    text = re.sub(r'[📋🏠🎁🚀🛎️ㆍ•·]', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 연속 줄바꿈을 공백으로
    text = re.sub(r'\n+', ' ', text)
    
    # 연속 공백 정리
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()


def extract_bullet_points(text):
    """불릿 포인트 항목 추출"""
    # 다양한 불릿 포인트 패턴
    patterns = [
        r'[-−–—]\s*(.+?)(?=[-−–—]|\n\n|$)',
        r'[•∙·]\s*(.+?)(?=[•∙·]|\n\n|$)',
        r'\d+[.)]\s*(.+?)(?=\d+[.)]|\n\n|$)',
    ]
    
    items = []
    for pattern in patterns:
        matches = re.findall(pattern, text, re.MULTILINE | re.DOTALL)
        for match in matches:
            cleaned = clean_text(match)
            if len(cleaned) > 5:  # 너무 짧은 것 제외
                items.append(cleaned)
    
    return items

def extract_section(markdown_text, section_name):
    """특정 섹션 추출"""
    patterns = section_patterns.get(section_name, [])
    
    for pattern in patterns:
        match = re.search(pattern, markdown_text, re.DOTALL | re.IGNORECASE)
        if match:
            content = match.group(1).strip()
            
            # 불릿 포인트 추출 시도
            items = extract_bullet_points(content)
            if items:
                return ', '.join(items)
            
            # 불릿 포인트가 없으면 전체 텍스트 정제
            return clean_text(content)
    
    return ""

def extract_all_sections(markdown_text):
    """모든 섹션 추출"""
    result = {}
    
    for section_name in ['주요업무', '자격요건', '우대사항', '채용절차', '복지 및 혜택', '급여']:
        result[section_name] = extract_section(markdown_text, section_name)
    
    return result

In [ ]:
driver = webdriver.Chrome()

detail_param = {
    "keyword":"ai 엔지니어",
    "title":"[주식회사 한울반도체]AI 엔지니어채용 (경력)",
    "company":"주식회사 한울반도체",
    "location":"경기 군포시",
    "career":"경력3년↑",
    "education":"초대졸↑",
    "work_type":"정규직",
    "deadline":"~ 02\/25(수)",
    "link":"https:\/\/www.saramin.co.kr\/zf_user\/jobs\/relay\/view?view_type=search&rec_idx=52917838&location=ts&searchword=ai+%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4&searchType=search&paid_fl=n&search_uuid=878295a2-fb7b-46bb-869b-11c2303efdda",
    "rec_idx":"52917838",
    "keyword_text":"AI(인공지능) S\/W Python AI(인공지능) DevOps",
    "crawled_at":"2026-01-27 17:09:48"
  }

def image_to_text(img_url):
    reader = easyocr.Reader(['ko', 'en'], gpu=False)
    result = reader.readtext(img_url, detail=0)    
    result_text = ""
    for text in result:
        result_text += text + " "

    print(result_text)
    return result_text

def detail_webdriver_search(detail_param):

    wait = WebDriverWait(driver, 20)
    link_url = f"https://m.saramin.co.kr/job-search/view?rec_idx=52760632"
    driver.get(link_url)    

    wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe[id^='iframe_recruit_detail_'], iframe[src*='view-frame']")))
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detailContentsWrap")))
    job_content = driver.find_element(By.CSS_SELECTOR, ".detailContentsWrap").text

    img_contents = driver.find_elements(By.CSS_SELECTOR, ".image > img")
    img_text = ""
    if len(img_contents) > 0:
        for img_content in img_contents:
            img_url=img_content.get_attribute("src")
            img_text = image_to_text(img_url)
            job_content += " " + img_text        

    driver.switch_to.default_content()  # 찾아가기 초기화
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".detail_corp")))

    company_content = driver.find_element(By.CSS_SELECTOR, ".detail_corp")
    company_items = company_content.find_elements(By.CSS_SELECTOR, ".detail_item > dd")[2].text
    content_dict = extract_all_sections(job_content)
    
    detail_param["responsibilities"] = content_dict["주요업무"]
    detail_param["qualifications"] = content_dict["자격요건"]
    detail_param["preferred_quali"] = content_dict["우대사항"]
    detail_param["hiring_process"] = content_dict["채용절차"]
    detail_param["benefits"] = content_dict["복지 및 혜택"]
    detail_param["salary"] = content_dict["급여"]
    detail_param["corp_domain"] = company_items
    detail_param["totaltext"] = job_content


    return(detail_param)






detail_webdriver_search(detail_param)

